In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# --- 1. Carga de Datos ---
# Asegura que la variable df_text esté definida
# Si esta línea da error, la variable no está en memoria.
df_text = pd.read_csv('TestReviews.csv', encoding='ISO-8859-1')
df_text.rename(columns={'class': 'sentiment'}, inplace=True)

# --- 2. Configuración y Descarga de NLTK ---
print("Configurando NLTK...")
try:
    # Descargar los recursos esenciales y el nuevo faltante (punkt_tab)
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    # ¡Agregamos el recurso que el error te pidió!
    nltk.download('punkt_tab', quiet=True) 
except Exception as e:
    print(f"Advertencia: Hubo un error en la descarga de NLTK: {e}")

# Definir la lista de Stop Words
stop_words = set(stopwords.words('english'))
print("Stop words y recursos cargados.")

# --- 3. Función de Limpieza de Texto ---
def limpiar_texto(texto):
    # a. Normalización: convertir a minúsculas
    texto = texto.lower()

    # b. Eliminación de Ruido: quitar puntuación y números (solo dejar a-z y espacios)
    texto = re.sub(r'[^a-z\s]', '', texto)

    # c. Tokenización (dividir en palabras)
    tokens = word_tokenize(texto)

    # d. Eliminación de Stop Words y palabras cortas (longitud <= 2)
    tokens_limpios = [
        word for word in tokens if word not in stop_words and len(word) > 2
    ]

    # Reconstruir el texto limpio
    return " ".join(tokens_limpios)

# --- 4. Aplicación y Verificación ---
df_text['review_clean'] = df_text['review'].apply(limpiar_texto)

# Inspeccionar el resultado comparando la reseña original con la limpia
print("\n--- ¡LIMPIEZA DE TEXTO EXITOSA! Comparación (Primera Fila) ---")
print("Original:", df_text['review'].iloc[0])
print("Limpia:", df_text['review_clean'].iloc[0])

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# 1. Definir X (features) y Y (target)
X = df_text['review_clean']
Y = df_text['sentiment']

# 2. Dividir los datos antes de vectorizar para evitar 'data leakage'
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 3. Inicializar y Entrenar el Vectorizador TF-IDF
# max_features limita el número de palabras únicas (columnas) para evitar un exceso de memoria.
tfidf = TfidfVectorizer(max_features=5000)

# Entrenar (fit) el vectorizador solo con los datos de ENTRENAMIENTO
X_train_vectorized = tfidf.fit_transform(X_train)

# Aplicar la transformación (transform) al conjunto de PRUEBA
X_test_vectorized = tfidf.transform(X_test)

# 4. Verificación de la Matriz Final
print("\n--- Verificación de la Vectorización ---")
print(f"Dimensiones de la matriz de entrenamiento (Filas: Reseñas, Columnas: Palabras): {X_train_vectorized.shape}")
print(f"Número de características (palabras) usadas: {len(tfidf.get_feature_names_out())}")

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 1. Entrenar el Modelo Naive Bayes
print("\nEntrenando Modelo Naive Bayes...")
modelo_nb = MultinomialNB()
modelo_nb.fit(X_train_vectorized, Y_train)

# 2. Realizar Predicciones
Y_pred = modelo_nb.predict(X_test_vectorized)

# 3. Evaluación del Modelo
precision = accuracy_score(Y_test, Y_pred)

print("\n--- Resultados de la Clasificación ---")
print(f"✅ Precisión (Accuracy) del Modelo: {precision:.4f}")
print("\n--- Informe Detallado de Clasificación (Precisión, Recall, F1-Score) ---")
# El informe detallado es crucial para ver el rendimiento en cada clase (0=Negativo, 1=Positivo)
print(classification_report(Y_test, Y_pred, target_names=['Negativo (0)', 'Positivo (1)']))

# 1. Obtener los nombres de las 5000 palabras (features)
feature_names = np.array(tfidf.get_feature_names_out())

# 2. Obtener los logaritmos de las probabilidades para cada clase (0 y 1)
# np.exp() se usa para obtener el valor real de probabilidad, aunque el ranking se mantiene con el log.
probabilidades_clase_0 = modelo_nb.feature_log_prob_[0]
probabilidades_clase_1 = modelo_nb.feature_log_prob_[1]

# 3. Calcular el ratio de importancia: P(Palabra|Positivo) / P(Palabra|Negativo)
# Restar logaritmos equivale a dividir las probabilidades: log(P1/P0) = log(P1) - log(P0)
# Las palabras con mayor diferencia logarítmica (más positivas) son las más discriminantes.
ratio_log = probabilidades_clase_1 - probabilidades_clase_0

# 4. Crear un DataFrame para ordenar y analizar
df_impacto_palabras = pd.DataFrame({
    'Palabra': feature_names,
    'Ratio_Log': ratio_log
})

# 5. Obtener las 10 palabras más POSITIVAS (Ratio_Log más alto)
top_positivas = df_impacto_palabras.sort_values(by='Ratio_Log', ascending=False).head(10)

# 6. Obtener las 10 palabras más NEGATIVAS (Ratio_Log más bajo)
top_negativas = df_impacto_palabras.sort_values(by='Ratio_Log', ascending=True).head(10)


print("\n--- 🌟 Top 10 Palabras que Predicen Sentimiento POSITIVO ---")
print(top_positivas.round(4))
print("\n--- 💀 Top 10 Palabras que Predicen Sentimiento NEGATIVO ---")
print(top_negativas.round(4))